data was scrapped on june 13 2020

In [1]:
#Import Libraries
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None


## Import and Cleaning

In [2]:
calendar = pd.read_csv('D:/Desktop/Datasets/AirbnbBerlin/calendar.csv.gz');
listings1 = pd.read_csv('D:/Desktop/Datasets/AirbnbBerlin/listings.csv');
listings2 = pd.read_csv('D:/Desktop/Datasets/AirbnbBerlin/listings.csv.gz');
neighborhoods = pd.read_csv('D:/Desktop/Datasets/AirbnbBerlin/neighbourhoods.csv');
reviews1 = pd.read_csv('D:/Desktop/Datasets/AirbnbBerlin/reviews.csv');
reviews2 = pd.read_csv('D:/Desktop/Datasets/AirbnbBerlin/reviews.csv.gz');

#Data Notes:

#locations are not exact (0-450 feet from actual address) due to how airbnb displays data. Therefore a heatmap will work well
#price is in dollars

c:\users\alex cerce\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (43,61,62) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Start by fixing the data types for each table

In [3]:
#Data cleaning functions used below

#I would prefer integers to have no trailing zeros

#This function will search a specifc dataframe column and return the number of non integer values and NAN values
#I am doing this because I want to keep the trailing zeros if there are values in the array 
#I wont be able to take away trailing zeros if there are any nan values
def find_noninteger(Dataframe,feature): #feature should be input as a string to be consistent with normal pandas format
    value=Dataframe[feature];
    float_count=0;na_count=0;index_nonint=[];index_na=[];
    for i in range(len(value)):
        if (value[i]).is_integer()==0 | pd.isna(value[i])==0:
            float_count +=1
            index_nonint.append(i)
        if pd.isna(value[i])==1:
            na_count +=1
            #index_na.append(i)
    if float_count==1 and na_count==1:
        print('There is',float_count,'non integer value and',na_count,'NAN value')
    elif float_count==1 and na_count!=1:
        print('There is',float_count,'non integer value and',na_count,'NAN values')
    elif float_count!=1 and na_count==1:
        print('There are',float_count,'non integer values and',na_count,'NAN value')
    elif float_count!=1 and na_count!=1:
        print('There are',float_count,'non integer values and',na_count,'NAN values')
    if index_nonint or index_na:
        print('The non integer values are located at:',index_nonint)

### Calendar Data Table
    The calendar table holds availibility data for every listing. The availibility is given from
    the date sampled up to one full year. Therefore, each listing will have 365 sample points.

In [4]:
calendar.head(n=5)

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,80843,2020-06-14,f,$39.00,$39.00,2.0,1125.0
1,186851,2020-06-15,f,$33.00,$33.00,62.0,300.0
2,186851,2020-06-16,f,$33.00,$33.00,62.0,300.0
3,186851,2020-06-17,f,$33.00,$33.00,62.0,300.0
4,186851,2020-06-18,f,$33.00,$33.00,62.0,300.0


In [5]:
#Missing Values
calendar_missingvalues_df=calendar[calendar.isna().any(axis=1)]    #dataframe containing only rows with any missing values
calendar_missingvalues_columns=calendar.columns[calendar.isna().any()].tolist()  #list of columns with missing values
print('These features have missing values:',calendar_missingvalues_columns)
print('\'minimum_nights\' has',calendar['minimum_nights'].isna().sum(),'missing values')
print('\'maximum_nights\' has',calendar['maximum_nights'].isna().sum(),'missing values')

# 'minimumn_nights' and 'maximum_nights':

#I have found that these 141 missing values are all from different listings, and they all
#occur on the sample date. This is just a data collection error
#I can pull the min and max nights values from other entries since they will be the same for the listing 

#pull the listing_id for the missing value, set missing values to be the same as the following row in full df
#use the sub table to reduce cpu load
for i in range(len(calendar_missingvalues_df)):
    missing_listing_id=calendar_missingvalues_df.iloc[i]['listing_id']
    missing_index=calendar_missingvalues_df.index[calendar_missingvalues_df['listing_id']==missing_listing_id].values[0]
    missing_listing_df=calendar.loc[calendar['listing_id']==missing_listing_id]
    
    calendar.at[missing_index,'minimum_nights']=missing_listing_df.iloc[50]['minimum_nights']  #choose arbitrary future date to set these values to
    calendar.at[missing_index,'maximum_nights']=missing_listing_df.iloc[50]['maximum_nights']
    
print('After filling missing values with the correct values:')
print('\'minimum_nights\' has',calendar['minimum_nights'].isna().sum(),'missing values')
print('\'maximum_nights\' has',calendar['maximum_nights'].isna().sum(),'missing values')


These features have missing values: ['minimum_nights', 'maximum_nights']
'minimum_nights' has 141 missing values
'maximum_nights' has 141 missing values
After filling missing values with the correct values:
'minimum_nights' has 0 missing values
'maximum_nights' has 0 missing values


In [6]:
#check columns for decimal values. This will indicated whether or not I should remove trailing zeros (by converting to INT)
print('minimum nights:')
#find_noninteger(calendar,'minimum_nights')
print('maximum nights:')
#find_noninteger(calendar,'maximum_nights')
print('-'*50);print('Before:');print(calendar.dtypes)  #This indicates, that once I deal with missing values I can convert these cols to int


#Convert Data Types
#convert to datetime
calendar['date']=calendar['date'].astype('datetime64')
#convert to boolean
bool_map={'t':True,'f':False}
calendar['available']=calendar['available'].map(bool_map).astype('bool')
#remove dollar signs, remove comas, and convert to float
calendar['price']=calendar['price'].str.replace('$','').str.replace(',','').astype('float64') 
calendar['adjusted_price']=calendar['adjusted_price'].str.replace('$','').str.replace(',','').astype('float64')
#Convert to INT (no decimal place values and no more missing values)
calendar['minimum_nights']=calendar['minimum_nights'].astype(int)
calendar['maximum_nights']=calendar['maximum_nights'].astype(int)
print('-'*50);print('After:')
print(calendar.dtypes)


minimum nights:
maximum nights:
--------------------------------------------------
Before:
listing_id          int64
date               object
available          object
price              object
adjusted_price     object
minimum_nights    float64
maximum_nights    float64
dtype: object
--------------------------------------------------
After:
listing_id                 int64
date              datetime64[ns]
available                   bool
price                    float64
adjusted_price           float64
minimum_nights             int32
maximum_nights             int32
dtype: object


In [7]:
calendar.head()

#number of observations in df
#number of unique listings

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,80843,2020-06-14,False,39.0,39.0,2,1125
1,186851,2020-06-15,False,33.0,33.0,62,300
2,186851,2020-06-16,False,33.0,33.0,62,300
3,186851,2020-06-17,False,33.0,33.0,62,300
4,186851,2020-06-18,False,33.0,33.0,62,300


### Listings Data Table

##### Listings_1 Data Table
    This table provides a summary of every available listing with basic, important information

In [8]:
listings1.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2015,Berlin-Mitte Value! Quiet courtyard/very central,2217,Ion,Mitte,Brunnenstr. Süd,52.53454,13.40256,Entire home/apt,60,4,132,2020-04-03,2.59,7,180
1,3176,Fabulous Flat in great Location,3718,Britta,Pankow,Prenzlauer Berg Südwest,52.53500,13.41758,Entire home/apt,90,62,146,2020-05-27,1.09,1,0
2,3309,BerlinSpot Schöneberg near KaDeWe,4108,Jana,Tempelhof - Schöneberg,Schöneberg-Nord,52.49885,13.34906,Private room,28,7,27,2019-05-31,0.32,1,364
3,7071,BrightRoom with sunny greenview!,17391,BrightRoom,Pankow,Helmholtzplatz,52.54316,13.41509,Private room,33,1,293,2020-03-31,2.22,2,61
4,9991,Geourgeous flat - outstanding views,33852,Philipp,Pankow,Prenzlauer Berg Südwest,52.53303,13.41605,Entire home/apt,180,6,8,2020-01-04,0.14,1,10


In [9]:
#change col name id to listing_id
listings1.rename(columns={'id':'listing_id','neighbourhood':'neighborhood'},inplace=True)
#remove 'neighbourhood_group' this is already shown in neighborhood table, also remove last_review since I don't care when a listing was last_reviewed
listings1.drop(['neighbourhood_group','last_review'],axis=1,inplace=True)

print(listings1.dtypes)

listing_id                          int64
name                               object
host_id                             int64
host_name                          object
neighborhood                       object
latitude                          float64
longitude                         float64
room_type                          object
price                               int64
minimum_nights                      int64
number_of_reviews                   int64
reviews_per_month                 float64
calculated_host_listings_count      int64
availability_365                    int64
dtype: object


In [10]:
print('This table includes basic listing infromation for',listings1.shape[0],'Airbnbs')

This table includes basic listing infromation for 24813 Airbnbs


##### Listings_2 Data Table
    This table provides a much more indepth version of Listings_1. This table includes 106 columns, many of them will not be used.
    I will take out columns that i feel will have no impact on the price of the apartment or that will be useful to see in visualizations.
    Some examples of things I will remove are urls,repeated info,and unneeded ID info.
    I will also remove information which is already shown in Listings_1 since I don't need the repeated information. However, 'listing_id' will
    be left there since it is acting as my primary key
    
    I will use this table when I want more information for a listing which Listings_1 doesn't provide

In [11]:
#listings2.head()

In [12]:
experiences_offered=listings2.loc[listings2['experiences_offered']!='none']
print(len(experiences_offered),'Airbnbs offer and additional experience')   #therfore will remove this col

listings2.rename(columns={'id':'listing_id','neighbourhood':'neighborhood'},inplace=True)
listings2.drop(['scrape_id','last_scraped','experiences_offered','thumbnail_url','medium_url','picture_url','xl_picture_url','host_url','host_location','host_about','host_thumbnail_url','host_picture_url',
                'host_neighbourhood','host_has_profile_pic','city','state','zipcode','market','smart_location','country_code','country','is_location_exact','minimum_minimum_nights','maximum_minimum_nights',
                'minimum_maximum_nights','maximum_maximum_nights','minimum_nights_avg_ntm','maximum_nights_avg_ntm','calendar_last_scraped','number_of_reviews_ltm','first_review','last_review','review_scores_accuracy',
                'review_scores_cleanliness','review_scores_checkin','review_scores_communication','review_scores_location','review_scores_value','requires_license','license','jurisdiction_names',
                'require_guest_profile_picture','require_guest_phone_verification','calculated_host_listings_count_entire_homes','calculated_host_listings_count_private_rooms','calculated_host_listings_count_shared_rooms',
                'street','neighbourhood_cleansed','neighbourhood_group_cleansed','host_total_listings_count','is_business_travel_ready','host_verifications','host_since','listing_url',
                'host_id','host_listings_count','host_identity_verified','neighborhood','latitude','longitude','room_type', 'price', 'minimum_nights','last_review','reviews_per_month','number_of_reviews',
                'calculated_host_listings_count'],axis=1,inplace=True)
print(listings2.columns.values)
#I have left only what I think may be useful

0 Airbnbs offer and additional experience
['listing_id' 'name' 'summary' 'space' 'description'
 'neighborhood_overview' 'notes' 'transit' 'access' 'interaction'
 'house_rules' 'host_name' 'host_response_time' 'host_response_rate'
 'host_acceptance_rate' 'host_is_superhost' 'property_type' 'accommodates'
 'bathrooms' 'bedrooms' 'beds' 'bed_type' 'amenities' 'square_feet'
 'weekly_price' 'monthly_price' 'security_deposit' 'cleaning_fee'
 'guests_included' 'extra_people' 'maximum_nights' 'calendar_updated'
 'has_availability' 'availability_30' 'availability_60' 'availability_90'
 'availability_365' 'review_scores_rating' 'instant_bookable'
 'cancellation_policy']


In [13]:
print('This table includes',listings2.shape[1],'types of listing infromation for',listings2.shape[0],'Airbnbs')
print('');print('Important features in the Listings_2 data table:')
print('host_is_superhost')
print('accommodates')
print('bathrooms')
print('bedrooms')
print('beds')
print('square_feet')
print('reviews_scores_rating')
print('maximum_nights')
print('availability_30')


This table includes 40 types of listing infromation for 24813 Airbnbs

Important features in the Listings_2 data table:
host_is_superhost
accommodates
bathrooms
bedrooms
beds
square_feet
reviews_scores_rating
maximum_nights
availability_30


#### Listings Data Table
    I will move features in Listings_2 which I think are important to Listings_1 which I will rename as Listings.
    This will give me a main Listings data table from which I can query the most important listing information. Listings_2
    will then be a a data table for extra information if I want more indepth knowledge on the listings

In [14]:
listings=listings1
listings2_important=['square_feet','review_scores_rating','host_is_superhost','accommodates','bathrooms','bedrooms','beds','maximum_nights','availability_30']
for feature in listings2_important:
    listings[feature]=listings2[feature]
    listings2.drop([feature],axis=1,inplace=True)

In [15]:
#name and host_name will be removed and kept in listings2
listings.drop(['name','host_name'],axis=1,inplace=True)
#make superhost col boolean
listings['host_is_superhost']=listings['host_is_superhost'].map(bool_map).astype('bool')
#rearrange cols so that similar features are near each other
listings_cols=listings.columns.tolist();lc=listings_cols
listings_cols=lc[:7]+[lc[12]]+[lc[8]]+[lc[9]]+lc[13:19]+[lc[10]]+[lc[7]]+[lc[19]]+[lc[20]]+[lc[11]]
listings=listings[listings_cols]

In [16]:
#missing values
listings_missingvalues_df=listings[listings.isna().any(axis=1)]    #dataframe containing only rows with any missing values
listings_missingvalues_columns=listings.columns[listings.isna().any()].tolist()  #list of columns with missing values
print('These features have missing values:',listings_missingvalues_columns)
print('\'square_feet\' has',listings['square_feet'].isna().sum(),'missing values')
print('\'reviews_per_month\' has',listings['reviews_per_month'].isna().sum(),'missing values')
print('\'review_scores_rating\' has',listings['review_scores_rating'].isna().sum(),'missing values')
print('\'bathrooms\' has',listings['bathrooms'].isna().sum(),'missing values')
print('\'bedrooms\' has',listings['bedrooms'].isna().sum(),'missing values')
print('\'beds\' has',listings['beds'].isna().sum(),'missing values')

These features have missing values: ['square_feet', 'reviews_per_month', 'review_scores_rating', 'bathrooms', 'bedrooms', 'beds']
'square_feet' has 24414 missing values
'reviews_per_month' has 4670 missing values
'review_scores_rating' has 5167 missing values
'bathrooms' has 20 missing values
'bedrooms' has 42 missing values
'beds' has 222 missing values


In [17]:
print('This table includes',listings.shape[1],'types of listing infromation for',listings.shape[0],'Airbnbs')
print(listings.dtypes)
listings.head()

This table includes 21 types of listing infromation for 24813 Airbnbs
listing_id                          int64
host_id                             int64
neighborhood                       object
latitude                          float64
longitude                         float64
room_type                          object
price                               int64
square_feet                       float64
number_of_reviews                   int64
reviews_per_month                 float64
review_scores_rating              float64
host_is_superhost                    bool
accommodates                        int64
bathrooms                         float64
bedrooms                          float64
beds                              float64
calculated_host_listings_count      int64
minimum_nights                      int64
maximum_nights                      int64
availability_30                     int64
availability_365                    int64
dtype: object


,listing_id,host_id,neighborhood,latitude,longitude,room_type,price,square_feet,number_of_reviews,reviews_per_month,...,host_is_superhost,accommodates,bathrooms,bedrooms,beds,calculated_host_listings_count,minimum_nights,maximum_nights,availability_30,availability_365
0,2015,2217,Brunnenstr. Süd,52.53454,13.40256,Entire home/apt,60,NaN,132,2.59,...,False,3,1.0,1.0,0.0,7,4,1125,30,180
1,3176,3718,Prenzlauer Berg Südwest,52.53500,13.41758,Entire home/apt,90,720.0,146,1.09,...,False,4,1.0,1.0,2.0,1,62,1125,0,0
2,3309,4108,Schöneberg-Nord,52.49885,13.34906,Private room,28,0.0,27,0.32,...,False,1,1.0,1.0,1.0,1,7,35,29,364
3,7071,17391,Helmholtzplatz,52.54316,13.41509,Private room,33,NaN,293,2.22,...,True,2,1.0,1.0,2.0,2,1,10,0,61
4,9991,33852,Prenzlauer Berg Südwest,52.53303,13.41605,Entire home/apt,180,NaN,8,0.14,...,False,7,2.5,4.0,7.0,1,6,14,10,10


### Reviews Data Table
* changed name from reviews_2 since I am not using table reviews_1

##### Reviews_1 Data Table

In [18]:
#This is just a subset of the Reviews2 data table. Therefore I will not use this
reviews1.head()

,listing_id,date
0,2015,2016-04-11
1,2015,2016-04-15
2,2015,2016-04-26
3,2015,2016-05-10
4,2015,2016-05-14


##### Reviews Data Table

In [19]:
reviews2.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,2015,69544350,2016-04-11,7178145,Rahel,Mein Freund und ich hatten gute gemütliche vie...
1,2015,69990732,2016-04-15,41944715,Hannah,Jan was very friendly and welcoming host! The ...
2,2015,71605267,2016-04-26,30048708,Victor,Un appartement tres bien situé dans un quartie...
3,2015,73819566,2016-05-10,63697857,Judy,"It is really nice area, food, park, transport ..."
4,2015,74293504,2016-05-14,10414887,Romina,"Buena ubicación, el departamento no está orden..."


In [20]:
#rename to reviews (no longer using reviews1)
reviews=reviews2
#change date to datetime
reviews['date']=reviews['date'].astype('datetime64')
#change col 'id' to 'review_id'
reviews.rename(columns={'id':'review_id'},inplace=True)
#drop reviewer name since it is not really needed
reviews.drop(['reviewer_name'],axis=1,inplace=True)
print(reviews.dtypes)


listing_id              int64
review_id               int64
date           datetime64[ns]
reviewer_id             int64
comments               object
dtype: object


In [21]:
#missing values
reviews_missingvalues_df=reviews[reviews.isna().any(axis=1)]    #dataframe containing only rows with any missing values
reviews_missingvalues_columns=reviews.columns[reviews.isna().any()].tolist()  #list of columns with missing values
print('These features have missing values:',reviews_missingvalues_columns)
print('\'comments\' has',reviews['comments'].isna().sum(),'missing values out of',reviews.shape[0],'total reviews for all Airbnbs')

These features have missing values: ['comments']
'comments' has 327 missing values out of 516939 total reviews for all Airbnbs


### Neighborhoods Data Table

In [22]:
#Table which indicates the borough which each neighborhood belongs to
neighborhoods.columns = ['borough', 'neighborhood']   #rename columns 
neighborhoods.head()

,borough,neighborhood
0,Charlottenburg-Wilm.,Barstraße
1,Charlottenburg-Wilm.,Charlottenburg Nord
2,Charlottenburg-Wilm.,Düsseldorfer Straße
3,Charlottenburg-Wilm.,Forst Grunewald
4,Charlottenburg-Wilm.,Grunewald


# EDA

# Visualizations

# Pricing
* use xgboost regressor to find the the optimal listing price